In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

In [14]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/openai_result.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/test_eng.csv"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


File '/kaggle/working/openai_result.csv' already there; not retrieving.

File '/kaggle/working/test_eng.csv' already there; not retrieving.



In [15]:
df = pd.read_csv('/kaggle/working/openai_result.csv')
df

id  \
0     eng_train_track_a_00001   
1                         NaN   
2                         NaN   
3     eng_train_track_a_00002   
4                         NaN   
...                       ...   
8302                      NaN   
8303                      NaN   
8304                      NaN   
8305                      NaN   
8306                      NaN   

                                                   text  Anger  Fear  Joy  \
0                                   But not very happy.      0     0    1   
1                           I was overwhelmed with joy.      0     0    1   
2                          My heart ached with sadness.      0     0    1   
3     Well she's not gon na last the whole song like...      0     0    1   
4     I can't help but feel a rush of joy as I suppo...      0     0    1   
...                                                 ...    ...   ...  ...   
8302  This situation is absolutely infuriating and h...      1     0    0   
8303  My heart raced as the spark jumped from his bo...      0     1    0   
8304  I gasped in surprise as the spark traveled fro...      0     1    0   
8305  I never expected to win against someone with 4...      0     0    1   
8306  Despite being outsized, I was filled with joy ...      0     0    1   

      Sadness  Surprise  
0           1         0  
1           1         0  
2           1         0  
3           0         0  
4           0         0  
...       ...       ...  
8302        1         0  
8303        0         1  
8304        0         1  
8305        0         1  
8306        0         1  

[8307 rows x 7 columns]

In [16]:
df_test = pd.read_csv('/kaggle/working/test_eng.csv')
df_test

id                                               text  \
0    eng_dev_track_a_00001            My mouth fell open `` No, no, no... I..   
1    eng_dev_track_a_00002  You can barely make out your daughter's pale f...   
2    eng_dev_track_a_00003  But after blinking my eyes for a few times lep...   
3    eng_dev_track_a_00004  Slowly rising to my feet I came to the conclus...   
4    eng_dev_track_a_00005  I noticed this months after moving in and doin...   
..                     ...                                                ...   
111  eng_dev_track_a_00112                       "ARcH stop your progression.   
112  eng_dev_track_a_00113        This 'star', starts to move across the sky.   
113  eng_dev_track_a_00114                                  and my feet hurt.   
114  eng_dev_track_a_00115        so i cried my eyes out and did the drawing.   
115  eng_dev_track_a_00116                              They were coal black.   

     Anger  Fear  Joy  Sadness  Surprise  
0      NaN   NaN  NaN      NaN       NaN  
1      NaN   NaN  NaN      NaN       NaN  
2      NaN   NaN  NaN      NaN       NaN  
3      NaN   NaN  NaN      NaN       NaN  
4      NaN   NaN  NaN      NaN       NaN  
..     ...   ...  ...      ...       ...  
111    NaN   NaN  NaN      NaN       NaN  
112    NaN   NaN  NaN      NaN       NaN  
113    NaN   NaN  NaN      NaN       NaN  
114    NaN   NaN  NaN      NaN       NaN  
115    NaN   NaN  NaN      NaN       NaN  

[116 rows x 7 columns]

In [17]:
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Apply the clean_text function
df_test['text'] = df_test['text'].apply(clean_text)

# Step 3-5: (Same as your original code)
df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# Ensure emotion columns are integers
# emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df_test[emotion_columns] = df_test[emotion_columns]

# Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_test_english.csv'
df_test.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.
Total samples: 116

Emotion distribution in the dataset:
Anger       0.0
Fear        0.0
Joy         0.0
Sadness     0.0
Surprise    0.0
dtype: float64

First few processed examples:
                      id                                               text  \
0  eng_dev_track_a_00001               my mouth fell open no, no, no... i..   
1  eng_dev_track_a_00002  you can barely make out your daughter's pale f...   
2  eng_dev_track_a_00003  but after blinking my eyes for a few times lep...   
3  eng_dev_track_a_00004  slowly rising to my feet i came to the conclus...   
4  eng_dev_track_a_00005  i noticed this months after moving in and doin...   

   Anger  Fear  Joy  Sadness  Surprise  
0    NaN   NaN  NaN      NaN       NaN  
1    NaN   NaN  NaN      NaN       NaN  
2    NaN   NaN  NaN      NaN       NaN  
3    NaN   NaN  NaN      NaN       NaN  
4    NaN   NaN  NaN     

/tmp/ipykernel_30/2770580193.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [18]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
#emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_train_english.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.
Total samples: 8307

Emotion distribution in the dataset:
Anger       1003
Fear        4833
Joy         2022
Sadness     2637
Surprise    2519
dtype: int64

First few processed examples:
                                                text  Anger  Fear  Joy  \
0                                But not very happy.      0     0    1   
1                        I was overwhelmed with joy.      0     0    1   
2                       My heart ached with sadness.      0     0    1   
3  Well she's not gon na last the whole song like...      0     0    1   
4  I can't help but feel a rush of joy as I suppo...      0     0    1   

   Sadness  Surprise  
0        1         0  
1        1         0  
2        1         0  
3        0         0  
4        0         0  


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5, problem_type="multi_label_classification")
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6, problem_type="multi_label_classification")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load tokenizer and model for sequence classification
tokenizer_deberta = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model_deberta = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=5)  # Set num_labels according to your task

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
tokenizer_xlm_roberta = AutoTokenizer.from_pretrained("xlm-roberta-base")
model_xlm_roberta = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=5)

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from datasets import load_dataset, Dataset, DatasetDict

In [23]:
train_dataset = Dataset.from_pandas(df)

train_dataset

dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 6645
    })
    test: Dataset({
        features: ['text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 1662
    })
})


In [24]:
# Convert train_dataset and val_dataset back to Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Save to CSV
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)

print("CSV files have been saved.")


CSV files have been saved.


In [25]:
train_df.head()

text  Anger  Fear  Joy  \
0  The fear of losing my vision terrifies me, for...      0     1    0   
1  I was filled with anger and fear when I found ...      1     1    0   
2  The shadows in the darkness made my skin crawl...      0     1    0   
3  I felt a wave of fear wash over me as my ankle...      0     1    0   
4            Her hair a mess and her words strained.      0     1    0   

   Sadness  Surprise  
0        0         0  
1        1         1  
2        0         0  
3        1         0  
4        1         0

In [26]:
val_df.head()

text  Anger  Fear  Joy  \
0                   How do we escape this situation?      0     1    0   
1        I shuddered, my heart pounding in my chest.      0     1    0   
2     Don't be afraid, we'll figure it out together!      0     1    1   
3  Yikes, my heart skipped a beat at the unexpect...      0     1    1   
4                       I had another one coming up.      0     0    0   

   Sadness  Surprise  
0        0         1  
1        0         0  
2        0         0  
3        0         1  
4        0         0

In [30]:
cols = train_df.columns[1:]
cols

Index(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'], dtype='object')

In [31]:
training_labels = train_df[cols]
training_labels = training_labels.to_numpy()
training_labels = training_labels.astype(np.float32)

train_sentences = train_df.text.values.tolist()

validation_labels = val_df[cols]
validation_labels = validation_labels.to_numpy()
validation_labels = validation_labels.astype(np.float32)

validation_sentences = val_df.text.values.tolist()

In [32]:
train_encodings = tokenizer(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(validation_sentences, padding=True, truncation=True, max_length=512)

In [34]:
from datasets import Dataset
import torch
import numpy as np

train_dataset_new = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': training_labels
})

validation_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': validation_labels
})

In [40]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)

epochs = 2
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(df) * epochs)

training_arguments = TrainingArguments(
    output_dir="distilbert_base_model",
    num_train_epochs=8,  # Adjust the number of epochs as needed
    per_device_train_batch_size=16,  # Adjust based on your GPU memory
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="no",  # Do not save after each epoch
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    report_to=[],  # Disable W&B logging
    remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [41]:
#defining the metrics to be calculated during training
def multi_label_metrics(predictions, labels,):
    sigmoid = torch.nn.Sigmoid()
    predictions = sigmoid(torch.tensor(predictions))
    y_predictions = np.zeros(predictions.shape)
    y_true = labels
    y_predictions[np.where(predictions >= 0.5)] = 1
    accuracy = accuracy_score(y_true, y_predictions)
    f1 = f1_score(y_true, y_predictions, average='micro')
    auc = roc_auc_score(y_true, y_predictions, average='micro')

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc
    }

    return metrics

#defining the function to compute the metrics for each batch of data
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(preds, p.label_ids)

    return result

In [42]:
import wandb

# Initialize wandb with mode disabled
wandb.init(mode="disabled")

In [43]:
train_encodings_roberta = tokenizer_xlm_roberta(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings_roberta = tokenizer_xlm_roberta(validation_sentences, padding=True, truncation=True, max_length=512)

train_dataset_roberta = Dataset.from_dict({
    'input_ids': train_encodings_roberta['input_ids'],
    'attention_mask': train_encodings_roberta['attention_mask'],
    'labels': training_labels
})

validation_dataset_roberta = Dataset.from_dict({
    'input_ids': val_encodings_roberta['input_ids'],
    'attention_mask': val_encodings_roberta['attention_mask'],
    'labels': validation_labels
})


optimizer_roberta = AdamW(model_xlm_roberta.parameters(), lr=5e-5, eps=1e-8)
training_arguments_roberta = TrainingArguments(
    output_dir="roberta_base_model",
    num_train_epochs=8,  # Adjust the number of epochs as needed
    per_device_train_batch_size=8,  # Adjust based on your GPU memory
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Do not save after each epoch
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    report_to=[],  # Disable W&B logging
    remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
)

trainer_roberta = Trainer(
    model=model_xlm_roberta,
    args=training_arguments_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=validation_dataset_roberta,
    compute_metrics=compute_metrics
)

trainer_roberta.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; 

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=3328, training_loss=0.1975648917723447, metrics={'train_runtime': 1735.8326, 'train_samples_per_second': 30.625, 'train_steps_per_second': 1.917, 'total_flos': 3169011352720320.0, 'train_loss': 0.1975648917723447, 'epoch': 8.0})

In [44]:
model_xlm_roberta.save_pretrained("/kaggle/working/roberta_model")

In [45]:
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")

Cache emptied for GPU 0
Cache emptied for GPU 1


In [46]:
del model_xlm_roberta

In [47]:
del train_encodings_roberta
del val_encodings_roberta

In [48]:
# Tokenize training and validation data
train_encodings_deberta = tokenizer_deberta(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings_deberta = tokenizer_deberta(validation_sentences, padding=True, truncation=True, max_length=512)

# Create datasets
train_dataset_deberta = Dataset.from_dict({
    'input_ids': train_encodings_deberta['input_ids'],
    'attention_mask': train_encodings_deberta['attention_mask'],
    'labels': training_labels
})

validation_dataset_deberta = Dataset.from_dict({
    'input_ids': val_encodings_deberta['input_ids'],
    'attention_mask': val_encodings_deberta['attention_mask'],
    'labels': validation_labels
})

# Define optimizer, scheduler, and training arguments
optimizer_deberta = AdamW(model_deberta.parameters(), lr=5e-5, eps=1e-8)
training_arguments_deberta = TrainingArguments(
    output_dir="deberta-base-model",
    num_train_epochs=8,  # Adjust the number of epochs as needed
    per_device_train_batch_size=16,  # Adjust based on your GPU memory
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="no",  # Do not save after each epoch
    logging_dir='./logs',
    logging_steps=10,
    warmup_steps=500,
    weight_decay=0.01,
    report_to=[],  # Disable W&B logging
    remove_unused_columns=False,  # IMPORTANT: Prevents the Trainer from removing columns
)

# Define the trainer and train
trainer_deberta = Trainer(
    model=model_deberta,
    args=training_arguments_deberta,
    train_dataset=train_dataset_deberta,
    eval_dataset=validation_dataset_deberta,
    compute_metrics=compute_metrics
)

trainer_deberta.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; 

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1664, training_loss=1.3921709393079464, metrics={'train_runtime': 1162.7629, 'train_samples_per_second': 45.719, 'train_steps_per_second': 1.431, 'total_flos': 3278346396969600.0, 'train_loss': 1.3921709393079464, 'epoch': 8.0})

In [49]:
model_deberta.save_pretrained("/kaggle/working/deberta_model")

In [50]:
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")

Cache emptied for GPU 0
Cache emptied for GPU 1


In [51]:
del model_deberta

In [52]:
del train_encodings_deberta
del val_encodings_deberta

In [53]:
#defining the trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset_new,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1664, training_loss=0.16413877797402585, metrics={'train_runtime': 442.5949, 'train_samples_per_second': 120.11, 'train_steps_per_second': 3.76, 'total_flos': 1691813028416400.0, 'train_loss': 0.16413877797402585, 'epoch': 8.0})

In [54]:
model.save_pretrained("/kaggle/working/distilbert_model")

In [55]:
del model

In [56]:
del train_encodings
del val_encodings

In [57]:
del trainer

In [58]:
del trainer_deberta

In [59]:
del trainer_roberta

In [60]:
from transformers import DistilBertForSequenceClassification, DebertaV2ForSequenceClassification, XLMRobertaForSequenceClassification

# Load BERT model
model_bert = DistilBertForSequenceClassification.from_pretrained("/kaggle/working/distilbert_model/")

# Load DeBERTa model
model_deberta = DebertaV2ForSequenceClassification.from_pretrained("/kaggle/working/deberta_model/")

model_xlm_roberta = XLMRobertaForSequenceClassification.from_pretrained("/kaggle/working/roberta_model/")

# Set models to eval mode
model_bert.eval()
model_deberta.eval()
model_xlm_roberta.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [61]:
# Test data (assuming you have a DataFrame with a column 'text')
test_sentences = df_test['text'].tolist()

# Tokenize the test data
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings_deberta = tokenizer_deberta(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
test_encodings_roberta = tokenizer_xlm_roberta(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [64]:
# Move the encodings to the same device as the model (GPU if available, otherwise CPU)
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
model_bert.to(device)

# Pass the test data through the model to get predictions
input_ids_bert = test_encodings['input_ids'].to(device)
attention_mask_bert = test_encodings['attention_mask'].to(device)

input_ids_deberta = test_encodings_deberta['input_ids'].to(device)
attention_mask_deberta = test_encodings_deberta['attention_mask'].to(device)

input_ids_roberta = test_encodings_roberta['input_ids'].to(device)
attention_mask_roberta = test_encodings_roberta['attention_mask'].to(device)

# BERT Predictions
with torch.no_grad():
    outputs_bert = model_bert(input_ids_bert, attention_mask=attention_mask_bert)

del model_bert

model_deberta.to(device)
# DeBERTa Predictions
with torch.no_grad():
    outputs_deberta = model_deberta(input_ids_deberta, attention_mask=attention_mask_deberta)

del model_deberta

model_xlm_roberta.to(device)

with torch.no_grad():
    outputs_roberta = model_xlm_roberta(input_ids_roberta, attention_mask=attention_mask_roberta)

del model_xlm_roberta

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
probabilities_bert = sigmoid(outputs_bert.logits)
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")
probabilities_deberta = sigmoid(outputs_deberta.logits)
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")
probabilities_roberta = sigmoid(outputs_roberta.logits)
for device_id in range(torch.cuda.device_count()):
    torch.cuda.set_device(device_id)
    torch.cuda.empty_cache()
    print(f"Cache emptied for GPU {device_id}")

NameError: name 'model_bert' is not defined

In [65]:
# Average predictions from BERT and DeBERTa
averaged_probabilities = (probabilities_bert + probabilities_deberta + probabilities_roberta) / 2

# Convert to binary predictions with a threshold of 0.5
final_predictions = (averaged_probabilities > 0.5).int()


In [66]:
# Convert to a DataFrame
label_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
predicted_df_ensemble = pd.DataFrame(final_predictions.cpu().numpy(), columns=label_columns)

# Combine with the original test sentences
result_df_ensemble = pd.concat([df_test['text'], predicted_df_ensemble], axis=1)

# Display the results
print(result_df_ensemble.head())


                                                text  Anger  Fear  Joy  \
0               my mouth fell open no, no, no... i..      0     1    0   
1  you can barely make out your daughter's pale f...      0     1    0   
2  but after blinking my eyes for a few times lep...      0     0    1   
3  slowly rising to my feet i came to the conclus...      0     1    0   
4  i noticed this months after moving in and doin...      0     1    0   

   Sadness  Surprise  
0        1         1  
1        1         0  
2        0         1  
3        1         0  
4        0         1  


In [67]:
result_df_ensemble['id'] = [f"eng_dev_track_a_{str(i+1).zfill(5)}" for i in range(len(result_df_ensemble))]

# Step 4: Drop the original 'text' column
result_df_ensemble = result_df_ensemble.drop(columns=['text'])

# Step 5: Reorder the columns so 'id' comes first, followed by the emotion labels
result_df_ensemble = result_df_ensemble[['id', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Step 6: Save the final DataFrame as 'final_submission.csv' for submission
result_df_ensemble.to_csv('final_submission.csv', index=False)

# Display the first few rows of the final DataFrame
print(result_df_ensemble.head())

print("Submission file 'final_submission.csv' has been created.")

                      id  Anger  Fear  Joy  Sadness  Surprise
0  eng_dev_track_a_00001      0     1    0        1         1
1  eng_dev_track_a_00002      0     1    0        1         0
2  eng_dev_track_a_00003      0     0    1        0         1
3  eng_dev_track_a_00004      0     1    0        1         0
4  eng_dev_track_a_00005      0     1    0        0         1
Submission file 'final_submission.csv' has been created.
